In [17]:
import pennylane as qml
import numpy as np


def qrom(lookup_table):
    """A subroutine that applies gates to implement the qROM oracle. 

    This particular qROM should work with variable n-bit address register with
    m-bit data values at each location. Given an address register prepared in
    the address |a> = |a0 a1 ... an> to query at, the qROM should write the
    contents of the address |b_a> = |b_a0 b_a1 ... b_am> to a data register,
    like so:
           _______
     a0 ---|     |--- a0
     a1 ---|  q  |--- a1
     .. ---|  R  |--- ..
     an ---|  O  |--- an
      0 ---|  M  |--- b_a0
     .. ---|     |--- ..
      0 ---|     |--- b_am
           -------

    Args:
        lookup_table (Dict[str, str]): The contents of the memory, in the form
            {address: contents} where address is an n-bit string, and contents is
            an m-bit string (see examples in test cases).
    """
    # YOUR CODE HERE
    print(lookup_table.keys())
    k = list(lookup_table.keys())[0]
    n = len(k)
    m = len(lookup_table[k])

    for k, v in lookup_table.items():
        for i in range(m):
            if int(v[i]):
                vals = [int(x) for x in k]
                qml.ctrl(qml.PauliX, list(range(n)), control_values=vals)(wires=n+i)


def problem_3(lookup_table):
    """Implement a quantum lookup table, or qROM.

    A few things to note:
        - the same QNode will be used for any address in the lookup table; only a different
          address will be passed each time
        - some addresses may not have any data stored at them; consider
          |b_a> = |0...0> for this case.
        - you can assume there will be no repeated addresses in the lookup table

    Args:
        lookup_table (Dict[str, str]): The contents of the memory, in the form
            {address: contents} where address is an n-bit string, and contents is
            an m-bit string (see examples in test cases).

    Returns:
        qml.QNode: Your qROM QNode. The QNode itself should return a single
        sample obtained from measuring the data wires.
    """

    # YOUR CODE HERE: create a device of appropriate size, with shots=1
    k = list(lookup_table.keys())[0]
    n = len(k)
    m = len(lookup_table[k])
    dev = qml.device('default.qubit', wires=n+m, shots=1)

    @qml.qnode(dev)
    def query_qrom(address=None):
        """Query the qROM at the provided address.

        This QNode should make use of the qROM oracle you implemented above.

        Args:
            address (str): An n-bit string indicating the address to query at.

        Returns:
            array[int]: Results of measuring a single sample of the data wires
            (corresponds to the content of the qROM at the input address). The
            length of the sample should be m.
        """
        # YOUR CODE HERE
        for i in range(n):
            if int(address[i]):
                qml.PauliX(wires=i)

        qrom(lookup_table=lookup_table)

        return qml.sample(wires=range(n, n+m, 1))

    return query_qrom

In [18]:
lookup_table = {
    "001": "01",
    "100": "10"
}

dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def test():
    problem_3(lookup_table=lookup_table)
    return qml.state()

# problem_3(lookup_table)
qrom(lookup_table=lookup_table)


dict_keys(['001', '100'])
